In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import h5py
import numpy as np
from faker import Factory
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import IntSlider, Output
import plotly.graph_objects as go
import plotly.express as plot_x
import matplotlib.colors as mcolors

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.reconstructions import reconstruction
from pyccapt.calibration.leap_tools import ccapt_tools

In [2]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [4]:
button_r = widgets.Button(
    description='load range dataset',
)
@button_r.on_click
def open_file_on_click_r(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global range_path
    range_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load range dataset', style=ButtonStyle())

In [5]:
tdc, pulse_mode, flightPathLength, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, flightPathLength)

Dropdown(description='Data mode:', options=('surface_concept', 'roentdec', 'leap_epos', 'leap_pos', 'ato_v6'),…

FloatText(value=110.0, description='Flight path length:')

In [7]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path 
variables.result_data_name = 'reconstruction_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/reconstruction/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength.value, max_mc.value)

The maximum time of flight: 5010


In [8]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,27.507521,27.400991,5263.160645,5263.160645,24501,607.555190,629.290080,-0.544408,-1.117469,0,0
1,0.0,0.0,0.0,27.031778,27.451994,5263.160645,5263.160645,25308,602.832214,642.745476,-0.977388,2.480082,807,2
2,0.0,0.0,0.0,26.986040,27.008272,5263.160645,5263.160645,25411,602.375972,625.195854,0.780000,0.977388,103,1
3,0.0,0.0,0.0,27.201569,27.270453,5263.160645,5263.160645,25674,604.522539,636.964182,0.971020,2.120327,263,1
4,0.0,0.0,0.0,27.571166,26.977407,5263.160645,5263.160645,26234,608.183934,642.718044,2.072571,-2.247673,560,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864263,0.0,0.0,0.0,27.122047,26.662453,5710.793945,5710.793945,24534,603.731533,613.146348,1.579102,-2.365469,2,1
1864264,0.0,0.0,0.0,27.270382,27.800477,5710.793945,5710.793945,24644,605.206103,620.381538,-2.091673,1.286204,110,1
1864265,0.0,0.0,0.0,27.024030,27.405944,5710.793945,5710.793945,24774,602.754954,609.010974,-1.505878,0.684490,130,1
1864266,0.0,0.0,0.0,26.964472,26.813825,5710.793945,5710.793945,25014,602.160690,623.460780,2.747510,2.142612,240,1


In [27]:
if 'range_path' in locals():
    variables.range_data = data_tools.read_hdf5_through_pandas(range_path)

else:
    variables.range_data = pd.DataFrame({"ion": ['unranged'], "mass": [0], "mc": [0], "mass": [0], "mc_low": [0], "mc_up": [0], "color": ['#000000'], "element": ['unranged'],
                                        "complex": [0], "isotop": [0], "charge": [0]})
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,peak_count,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,1.581404,1.357690,1.886822,#b2aa2d,163.000000,H,1,1,+
1,${}^{27}Al^{2+}$,13.490000,13.505377,12.998589,14.321418,#e7e0d1,64219.000000,Al,1,27,2+
2,${}^{27}Al^{+}$,26.980000,26.993149,25.856491,28.660889,#e7e0d1,589415.000000,Al,1,27,+


In [11]:
change_color = widgets.Button(
    description='change color',
)
color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
@change_color.on_click
def change_color_m(b,):
    with out:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
        
# Create a container for the widgets
container = widgets.VBox([color_picker, row_index, change_color])
out = Output()
# Display the widgets
display(container, out)

Output()

$$\textbf{You can specify which dataset to use in below block}$$

In [29]:
# romove data that has x_det and y_det equal to zero at same time
# these points resulted in nan in the reconstruction becuse of dviding 0/0
# mask = np.logical_and((data['x_det (cm)'].to_numpy()==0), (data['y_det (cm)'].to_numpy()==0))
# print('The number of data with x=0 and y=0 in dataset is:', len(mask[mask==True]))
# data.drop(np.where(mask)[0], inplace=True)
# data.reset_index(inplace=True, drop=True)

The number of data with x=0 and y=0 in dataset is: 1


In [30]:
# exctract needed data from Pandas data frame as an numpy array
# dld_highVoltage = data['high_voltage (V)'].to_numpy()
# dld_x = data['x_det (cm)'].to_numpy()
# dld_y = data['y_det (cm)'].to_numpy()
# mc = data['mc_c (Da)'].to_numpy()

In [28]:
def mc_hist_plot(bin_size, lim_mc, hist_color_range, figname):
    peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(variables.mc_c[variables.mc_c < lim_mc], variables, bin=bin_size, range_data=variables.range_data, ranging=True, hist_color_range=hist_color_range, adjust_label=False, peaks_find=True, peaks_find_plot=True, plot=True, label='mc', fig_name=figname)
   
interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)])
                , figname=widgets.Text(value='mc'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_38724\4082651408.py:4: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=400, description='lim_mc'), …

In [16]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [29]:
avg_dens = element_selected.value[2]
field_evap = element_selected.value[3]

In [48]:
if variables.range_data.empty:
    element_percentage = str([0.01])
else:
    element_percentage = [0.01] * len(variables.range_data['element'].tolist())
    element_percentage = str(element_percentage)
    

interact_manual(reconstruction.x_y_z_calculation_and_plot, kf=widgets.FloatText(value=4), det_eff=widgets.FloatText(value=0.7), icf=widgets.FloatText(value=1.4),
                field_evap=widgets.FloatText(value=field_evap),
               avg_dens=widgets.FloatText(value=avg_dens), element_percentage=widgets.Textarea(value=element_percentage),
               rotary_fig_save=widgets.Dropdown(options=[('True', True), ('False', False)], value=False), variables=fixed(variables), 
                flight_path_length=fixed(flightPathLength.value), selected_are=fixed(False), figname=widgets.Text(value='3d'),
               mode=widgets.Dropdown(options=[('Gault', 'Gault'), ('Bas', 'Bas')]));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_38724\3364758392.py:8: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



interactive(children=(Textarea(value='[0.01, 0.01, 0.01]', description='element_percentage'), FloatText(value=…

In [49]:
data['x (nm)'] = variables.x
data['y (nm)'] = variables.y
data['z (nm)'] = variables.z
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,-4.755963,-9.762240,0.856692,27.507521,27.400991,5263.160645,5263.160645,24501,607.555190,629.290080,-0.544408,-1.117469,0,0
1,-8.278379,21.006049,3.784047,27.031778,27.451994,5263.160645,5263.160645,25308,602.832214,642.745476,-0.977388,2.480082,807,2
2,6.813378,8.537580,0.866882,26.986040,27.008272,5263.160645,5263.160645,25411,602.375972,625.195854,0.780000,0.977388,103,1
3,8.299763,18.123417,2.930872,27.201569,27.270453,5263.160645,5263.160645,25674,604.522539,636.964182,0.971020,2.120327,263,1
4,17.344265,-18.809602,4.899756,27.571166,26.977407,5263.160645,5263.160645,26234,608.183934,642.718044,2.072571,-2.247673,560,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864263,14.435495,-21.624138,16.808484,27.122047,26.662453,5710.793945,5710.793945,24534,603.731533,613.146348,1.579102,-2.365469,2,1
1864264,-19.336231,11.890163,15.677703,27.270382,27.800477,5710.793945,5710.793945,24644,605.206103,620.381538,-2.091673,1.286204,110,1
1864265,-14.178133,6.444606,13.798764,27.024030,27.405944,5710.793945,5710.793945,24774,602.754954,609.010974,-1.505878,0.684490,130,1
1864266,24.584285,19.171754,18.939677,26.964472,26.813825,5710.793945,5710.793945,25014,602.160690,623.460780,2.747510,2.142612,240,1


In [70]:
plot_3d_button = widgets.Button(
    description='plot 3D',
)
plot_heatmap_button = widgets.Button(
    description='plot heatmap',
)
plot_mc_button = widgets.Button(
    description='plot mc',
)
plot_projection_button = widgets.Button(
    description='plot projection',
)

clear_button = widgets.Button(
    description='Clear plots',
)

figname_3d=widgets.Text(value='3d_plot', description='fig name')
selected_area_p3=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
rotary_fig_save_p3=widgets.Dropdown(options=[('True', True), ('False', False)], description='Rotary save')
element_percentage_p3=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_3d_button.on_click
def plot_3d(b, figname=figname):
    with out:
        if variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
        reconstruction.reconstruction_plot(variables, element_percentage_p3.value, rotary_fig_save_p3.value, selected_area_p3.value, figname_3d.value)

selected_area_ph=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
element_percentage_ph=widgets.Textarea(value=element_percentage, description='Element percentage')
figname_heatmap=widgets.Text(value='heatmap', description='fig name')
@plot_heatmap_button.on_click
def plot_heatmap(b, variables=variables):
    with out:
        if variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)

        reconstruction.heatmap(variables, selected_area_ph.value, element_percentage_ph.value, save=True)

selected_area_pm=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
range_file_exist_pm=widgets.Dropdown(options=[('True', True), ('False', False)], description='Range exist')
bin_size_pm = widgets.FloatText(value=0.1, description='Bins size')
lim_mc_pm = widgets.IntText(value=150, description='Limit mc')
hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)], description='hist color')
figname_mc=widgets.Text(value='mc', description='fig name')
@plot_mc_button.on_click
def plot_mc(b,):
    with out:
        if variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)

        peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(variables.mc_c[variables.mc_c < lim_mc_pm.value], variables, bin=bin_size_pm.value, range_data=variables.range_data, 
                                                               ranging=True, hist_color_range=hist_color_range.value, adjust_label=False, peaks_find=True, peaks_find_plot=True, plot=True, label='mc', fig_name=figname_mc.value)


element_percentage_pp=widgets.Textarea(value=element_percentage, description='Element percentage')
selected_area_pp=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
x_or_y_pp=widgets.Dropdown(options=['x', 'y'], value='x', description='X or Y')
figname_p = widgets.Text(value='projection', description='fig name')
@plot_projection_button.on_click
def plot_projection(b,):
    with out:
        if selected_area_pp.value:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)

        reconstruction.projection(variables, element_percentage_pp.value, selected_area_pp.value, x_or_y_pp.value, figname_p.value)

@clear_button.on_click
def clear(b,):
    with out:
        clear_output(True)
        print('')


tab1 = VBox(children=[selected_area_pp, x_or_y_pp, element_percentage_pp, figname_p, plot_projection_button, clear_button])
tab2 = VBox(children=[selected_area_p3, rotary_fig_save_p3, element_percentage_p3, figname_3d, plot_3d_button, clear_button])
tab3 = VBox(children=[selected_area_pm, bin_size_pm, lim_mc_pm, hist_color_range, figname_mc, plot_mc_button, clear_button])
tab4 = VBox(children=[selected_area_ph, element_percentage_ph, figname_heatmap, plot_heatmap_button, clear_button])
# tab2 = VBox(children=[VBox(children=[HBox(children=[plot_3d_button, selected_area_p3, range_file_exist_p3, rotary_fig_save_p3, element_percentage_p3]),
#                                      HBox(children=[plot_heatmap_button, selected_area_ph, element_percentage_ph]),
#                                      HBox(children=[plot_mc_button, selected_area_pm, range_file_exist_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm]), 
#                                      HBox(children=[plot_projection_button, selected_area_pp, x_or_y_pp, element_percentage_pp])]), clear_button])


tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'projection')
tab.set_title(1, '3d plot')
tab.set_title(2, 'mc plot')
tab.set_title(3, 'heatmap plot')



display(VBox(children=[tab]))
out = Output()
display(out)

Output()

In [71]:
interact_manual(data_tools.save_data, data=fixed(data), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…